In [200]:
import os
import json
import pandas as pd
import traceback

In [201]:
from langchain.chat_models import ChatOpenAI

In [202]:
from dotenv import load_dotenv
load_dotenv()

True

In [203]:
KEY=os.getenv("openai_api_key")

In [224]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(
    openai_api_key=KEY,                # ✅ Ensure KEY is defined (e.g., from environment)
    model_name="gpt-3.5-turbo",        # ✅ Supported model
    temperature=0.3,                   # ✅ Low creativity, focused answers
    request_timeout=140,              # ✅ Reasonable timeout
    max_tokens=500,                   # ✅ Max response length
    max_retries=3,                    # ✅ Resilient to transient failures
    streaming=False                   # ✅ Safer for logs & debugging
)

In [205]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2


In [206]:
response_json = """
{
  "1": {
    "mcq": "multiple choice question",
    "options": {
      "a": "choice here",
      "b": "choice here",
      "c": "choice here",
      "d": "choice here"
    },
    "correct": "a"
  },
  "2": {
    "mcq": "multiple choice question",
    "options": {
      "a": "choice here",
      "b": "choice here",
      "c": "choice here",
      "d": "choice here"
    },
    "correct": "b"
  },
  "3": {
    "mcq": "multiple choice question",
    "options": {
      "a": "choice here",
      "b": "choice here",
      "c": "choice here",
      "d": "choice here"
    },
    "correct": "c"
  }
}
"""


In [207]:
TEMPLATE = """
Text: {text}

You are an expert MCQ maker. Given the above text, it is your job to
create a quiz of {number} multiple choice questions for {subject} students in a {tone} tone.

Make sure the questions are not repeated and check all the questions to be conforming to the text as well. 
Make sure to format your response like RESPONSE_JSON below and use it as a guide.

Ensure to make {number} MCQs.

### RESPONSE_JSON
{response_json}
"""


In [156]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
)

In [208]:
quiz_chain = LLMChain(
    llm=llm,
    prompt=quiz_generation_prompt,
    output_key="quiz",  # <--- important
    verbose=True
)

In [210]:
TEMPLATE2 = """
You are an expert English grammarian and writer. Given a Multiple Choice Quiz for {subject} students,
you need to evaluate the complexity of the questions and give a complete analysis of the quiz. 
Use at most 50 words for the analysis.

If the quiz is not aligned with the cognitive and analytical abilities of the students, update the quiz questions that need to be changed, 
and adjust the tone to perfectly fit the students.

Quiz_MCQs:
{quiz}

Check and update the above quiz as an expert English writer.
"""


In [211]:
quiz_evaluation_prompt = PromptTemplate(
    input_variables=["subject", "quiz"],
    template=TEMPLATE2
)

In [212]:
review_chain = LLMChain(
    llm=llm,  # Ensure 'llm' is defined earlier as your language model (e.g., OpenAI)
    prompt=quiz_evaluation_prompt,
    output_key="review",
    verbose=True
)

In [213]:
from langchain.chains import  SequentialChain
from langchain.llms import HuggingFaceHub

In [214]:
generator_evaluate_chain = SequentialChain(
    chains=[quiz_chain, review_chain],
    input_variables=["text", "number", "subject", "tone", "response_json"],
    output_variables=["quiz", "review"],
    verbose=True
)

In [215]:
file_path = r'C:\Users\DELL\mcqgenderater\data\data.txt'



In [216]:
import os

file_path = 'C:\\Users\\DELL\\mcqgenderater\\data\\data.txt'
if os.path.exists(file_path):
    with open(file_path, 'r') as file:
        TEXT = file.read()

In [217]:
print(TEXT)

A multi-agent system (MAS or "self-organized system") is a computerized system composed of multiple interacting intelligent agents.[1][2] Multi-agent systems can solve problems that are difficult or impossible for an individual agent or a monolithic system to solve.[3] Intelligence may include methodic, functional, procedural approaches, algorithmic search or reinforcement learning.[4] With advancements in large language models (LLMs), LLM-based multi-agent systems have emerged as a new area of research, enabling more sophisticated interactions and coordination among agents.[5]

Despite considerable overlap, a multi-agent system is not always the same as an agent-based model (ABM). The goal of an ABM is to search for explanatory insight into the collective behavior of agents (which do not necessarily need to be "intelligent") obeying simple rules, typically in natural systems, rather than in solving specific practical or engineering problems. The terminology of ABM tends to be used mor

In [218]:
##Serialize the Python dictionary into a JSON-formatted string
json_string = json.dumps(response_json)

In [219]:

NUMBER = 5
SUBJECT = "Multi-agent_system"
TONE = "Friendly"

In [226]:
from langchain.callbacks import get_openai_callback

with get_openai_callback() as cb:
    response = generator_evaluate_chain({
        "text": TEXT,
        "number": NUMBER,
        "subject": SUBJECT,
        "tone": TONE,
        "response_json": json.dumps(response_json)
    })



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text: A multi-agent system (MAS or "self-organized system") is a computerized system composed of multiple interacting intelligent agents.[1][2] Multi-agent systems can solve problems that are difficult or impossible for an individual agent or a monolithic system to solve.[3] Intelligence may include methodic, functional, procedural approaches, algorithmic search or reinforcement learning.[4] With advancements in large language models (LLMs), LLM-based multi-agent systems have emerged as a new area of research, enabling more sophisticated interactions and coordination among agents.[5]

Despite considerable overlap, a multi-agent system is not always the same as an agent-based model (ABM). The goal of an ABM is to search for explanatory insight into the collective behavior of agents (which do not necessarily need to be "intelligent") obeying simple rules, typically in natural systems, r

BadRequestError: Error code: 400 - {'error': {'message': 'invalid model ID', 'type': 'invalid_request_error', 'param': None, 'code': None}}

In [185]:
print(f"Total Tokens: {cb.total_tokens}")
print(f"Prompt Tokens: {cb.prompt_tokens}")
print(f"Completion Tokens: {cb.completion_tokens}")
print(f"Total Cost: {cb.total_cost}")

Total Tokens: 0
Prompt Tokens: 0
Completion Tokens: 0
Total Cost: 0.0


In [235]:
import openai
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "Hello!"}]
)

print(response["choices"][0]["message"]["content"])

APIRemovedInV1: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


In [191]:
#quiz = response.get("quiz")

In [192]:
#json.loads(quiz)

In [ ]:
quiz_table_data = []

# Assuming 'quiz' is the dictionary containing your quiz data
for key, value in quiz.items():
    mcq = value["mcq"]  # Get the question text
    options = ", ".join(f"({option}): {option_value}" for option, option_value in value["options"].items())  # Format options
    correct = value["correct"]  # Get the correct answer
    
    # Append to quiz_table_data as a dictionary
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

# Now quiz_table_data will contain the desired output
print(quiz_table_data)


In [ ]:
pd.Datafarame(quiz_table_data)

In [195]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'04_30_2025_20_11_48'